In [ ]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from util import prepare_df

In [ ]:
df = prepare_df(
    "../outputs/20240213_185829_utilisation_header.csv",
    "../outputs/20240213_185829_utilisation_statespace_explo.csv",
)

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
task_palette = sns.color_palette("deep", int(df["n_task"].unique().shape[0]))
sns.set_theme(style="white", palette="deep")

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(
    index="tid",
    columns="search_type",
    values=["duration_s", "n_visited", "schedulable", "nbt", "U"],
)
df_piv.columns = list(map(lambda x: "_".join(x), df_piv.columns))
df_piv["n_visted_ratio"] = df_piv["n_visited_ACBFS"] / df_piv["n_visited_BFS"]

df_piv.rename(columns={"nbt_BFS": "Number of tasks"}, inplace=True)
df_piv.rename(columns={"schedulable_ACBFS": "Schedulable"}, inplace=True)
df_piv.rename(columns={"U_ACBFS": "Utilisation"}, inplace=True)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(12, 5.5129))

ax_reduc = axs[0]
ax_sched = axs[1]

sns.lineplot(
    data=df_piv, x="Utilisation", y="n_visted_ratio", errorbar=None, ax=ax_reduc
)
sns.lineplot(data=df_piv, x="Utilisation", y="Schedulable", errorbar=None, ax=ax_sched)

ax_reduc.set_ylabel("State reduction ratio")
ax_sched.set_ylabel("Acceptance ratio")

f.savefig(
    f"./{datetime.now().strftime('%Y%m%d_%H%M%S')}-reduction-accpetance-utilisation.pdf"
)